In [1]:
import numpy as np
import pandas as pd

import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats import diagnostic

from scipy import stats
import scipy.fft as sf

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

pd.options.plotting.backend = "plotly" 

In [43]:
def ft(df, metric):
    N = df.shape[0]

    # Fourier transform
    fft = np.abs(sf.rfft(df[metric].to_numpy()))/N
    freqs = sf.rfftfreq(N, d=0.1)[::-1]

    return freqs, fft

# ***Channels DF***

In [3]:
channels_df = pd.read_csv('data/df_channels_en.tsv.gz', sep='\t', compression='infer')
channels_df.head()

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087


In [4]:
channels_df.groupby('category_cc').mean()

,subscribers_cc,videos_cc,subscriber_rank_sb,weights
category_cc,,,,
Autos & Vehicles,111158.375169,682.188664,396373.648043,7.680089
Comedy,432108.482347,287.109371,299991.221927,6.782554
Education,268202.635140,552.903883,327824.047033,6.868359
Entertainment,351383.375321,645.804148,323417.997255,6.903652
Film and Animation,228242.992727,325.526109,371171.466182,7.608686
Gaming,202022.438068,750.962568,375404.958000,7.745133
Howto & Style,233022.904842,392.432337,322855.754779,6.834346
Music,292134.635948,666.219518,366227.229524,7.420399
News & Politics,294673.487406,7844.407424,318111.243482,6.788842


In [5]:
px.scatter(channels_df.sample(1000), x='subscribers_cc', y='weights', log_x=True)

### *Interesting Subgroups*

In [6]:
top_10_channels = channels_df.sort_values(by='subscribers_cc', ascending=False).iloc[:10]
top_10_channels

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
55,Music,2006-03-13,UCq-Fj5jknLsUf-MWSy4_brA,T-Series,112139463,13839,102.0,2.087
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087
5,Entertainment,2007-01-15,UCIwFjwMjI0y7PDBVEO9-bkQ,Justin Bieber,46574085,134,12.0,2.087
7,Sports,2009-03-17,UCRijo3ddMTht_IHyNSNXpNQ,Dude Perfect,45800000,213,14.0,2.087
6,Music,2014-03-12,UCFFbwnve3yF62-tVXkTyHqg,Zee Music Company,43451109,4241,13.0,2.087
9,Music,2006-08-08,UC0C-w0YjGpqDXGB8IHb662A,Ed Sheeran,42000000,154,17.0,2.087


# ***Timeseries***

### *Global Analysis*

In [7]:
timeseries_df = pd.read_csv('data/df_timeseries_en.tsv.gz', sep='\t', compression='infer')
timeseries_df['datetime'] = pd.to_datetime(timeseries_df['datetime'])
timeseries_df.head()

,channel,category,datetime,views,delta_views,subs,delta_subs,videos,delta_videos,activity
0,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-03,2.024946e+05,0.000000,650.222222,0.000000,5,0,3
1,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-10,3.940857e+05,191591.111111,1046.000000,395.777778,6,1,1
2,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-17,8.353938e+05,441308.083333,1501.500000,455.500000,6,0,1
3,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-24,1.104577e+06,269183.250000,1750.000000,248.500000,6,0,0
4,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-31,1.284406e+06,179828.600000,2008.300000,258.300000,6,0,0


In [8]:
mean_timeseries = timeseries_df.groupby('datetime').mean().reset_index()
mean_timeseries.head()

,datetime,views,delta_views,subs,delta_subs,videos,delta_videos,activity
0,2015-01-05,1.445569e+07,0.000000,43823.500000,0.000000,710.000000,0.000000,10.000000
1,2015-01-12,7.999070e+07,159754.659722,115951.479630,126.204167,307.000000,0.833333,4.166667
2,2015-01-19,6.888411e+07,320650.937992,99979.879400,524.611146,263.428571,0.285714,2.000000
3,2015-01-26,6.926567e+07,433905.663195,100398.323796,445.512698,264.285714,0.857143,1.285714
4,2015-02-02,6.961967e+07,406342.946529,100821.772006,450.519048,265.428571,1.142857,2.142857


In [9]:
mean_timeseries.plot.line(x='datetime', y='delta_subs', log_y=True)

In [10]:
stat = 'subs'

N = mean_timeseries.shape[0]

# Fourier transform
fft = sf.rfft(mean_timeseries[stat].to_numpy())
freqs = sf.rfftfreq(N, d=0.1)

px.bar(x=freqs[::-1], y=abs(fft)/N, title=f'Mean {stat} FT')

### *Sample Analysis*

In [38]:
sample_df = pd.read_parquet('data/df_sample_timeseries.parquet')
sample_df.shape

(8624, 12)

In [92]:
categorized_sample_df = sample_df.groupby(['game_category', 'datetime']).mean().reset_index().groupby('game_category')

game_categories = list(categorized_sample_df.groups.keys())
N = len(game_categories)
print(f'{N} categories')

8 categories


In [81]:
def timeseries_correlations(df, feature, verbose=False):
    corrs = np.zeros(shape=(N,N))
    for i, (category_a, category_a_df) in enumerate(df):
        for j, (category_b, category_b_df) in enumerate(categorized_sample_df):   
            corr = stats.pearsonr(category_a_df[feature], category_b_df[feature])
            corrs[i,j] = round(corr[0], 2)
            if verbose:
                print('-'*30)
                print(f"{category_a}-{category_b}")
                print(f"Delta subs Corr. : {corr}")
    return corrs

def fourier_plot(df, feature):
    fig = make_subplots(rows=N, cols=2, subplot_titles=(feature, 'Fourier Transform'))

    for i, (category, category_df) in enumerate(df):
        # Timeseries Plot
        fig.add_trace(
            go.Scatter(x=pd.to_datetime(category_df.datetime), y=category_df[feature]),
            row=i+1, col=1
        )
        
        # Computing Fourier Transform
        freqs, fft = ft(category_df, feature)

        # FT Plot
        fig.add_trace(
            go.Bar(x=freqs, y=fft),
            row=i+1, col=2
        )

        fig.update_yaxes(title_text=f'{category}', row=i+1, col=1)


    fig.update_layout(height=2500, width=1000, title_text=f"{feature} timeseries and its FT", showlegend=False)
    fig.show()

##### ***Metric Analysis***

In [90]:
metric = 'delta_views'
fourier_plot(categorized_sample_df, metric)

In [91]:
fig = px.imshow(timeseries_correlations(categorized_sample_df, metric), 
                x=list(game_categories),
                y=list(game_categories),
                text_auto=True,
                title=f'Categories {metric} Correlation')

fig.show()